In [13]:
import torch
import numpy as np
import open3d as o3d
import copy
import laspy
import os.path as path
import os
from scipy.spatial.transform import Rotation

In [14]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])

In [15]:
LAS_FILE = '/data/318000_5809500.laz'
with laspy.open(LAS_FILE) as fp:
    print('Points:', fp.header.point_count)
    las = fp.read()
src_x, src_y, src_z = las.x, las.y, las.z
src_points = np.stack((src_x, src_y, src_y)).T
local_origin = np.min(src_points, axis=0)
src_points -= local_origin

Points: 5344661


In [16]:
def random_sample_rotation(rotation_factor):
    euler = np.random.uniform(-1, 1, 3) * np.pi * rotation_factor
    rotation = Rotation.from_euler('zyx', euler).as_matrix()
    return rotation

def random_translation(shift_factor):
    shift = np.random.uniform(-shift_factor, shift_factor, 3)
    return shift

def apply_transform(points, transform):
    rotation = transform[:3, :3]
    translation = transform[:3, 3]
    points = np.matmul(points, rotation.T) + translation
    return points

def get_transform_from_rotation_translation(rotation, translation):
    transform = np.eye(4)
    transform[:3, :3] = rotation
    transform[:3, 3] = translation
    return transform

def get_bbox(points):
    x, y = points[:, 0], points[:, 1]
    return np.min(x), np.max(x), np.min(y), np.max(y)

def filter_by_bbox(points, bbox):
    x, y = points[:, 0], points[:, 1]
    xmin, xmax, ymin, ymax = bbox
    return points[(x >= xmin) & (x <= xmax) & (y >= ymin) & (y <= ymax)]

def make_open3d_point_cloud(points, colors=None, normals=None):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    if colors is not None:
        pcd.colors = o3d.utility.Vector3dVector(colors)
    if normals is not None:
        pcd.normals = o3d.utility.Vector3dVector(normals)
    return pcd

In [17]:
# print(get_bbox(src_points))

# src_points_filtered = filter_by_bbox(src_points, [80, 100, 80, 100])

# len(src_points_filtered)

src_points_filtered = src_points

In [18]:
aug_rotation_factor = 0.1
aug_shift_factor = 2

aug_rotation = random_sample_rotation(aug_rotation_factor)
aug_translation = random_translation(aug_shift_factor)
aug_transform = get_transform_from_rotation_translation(aug_rotation, aug_translation)

local_origin = np.min(src_points_filtered, axis=0)
src_points = src_points_filtered - local_origin
ref_points = apply_transform(src_points, aug_transform)

len(src_points), len(ref_points)

(5344661, 5344661)

In [19]:
print(src_points.min(axis=0), src_points.max(axis=0))
print(ref_points.min(axis=0), ref_points.max(axis=0))

[0. 0. 0.] [499.999 499.999 499.999]
[  0.81869914 -54.23751737  -5.09594435] [571.23318576 566.36944132 417.99365604]


In [20]:
src = make_open3d_point_cloud(src_points)
ref = make_open3d_point_cloud(ref_points)

In [7]:
IDENTITY = np.eye(4)
draw_registration_result(src, ref, IDENTITY)
draw_registration_result(src, ref, aug_transform)

In [8]:
os.makedirs('/data/street', exist_ok=True)
np.save('/data/street/src.npy', src_points)
np.save('/data/street/ref.npy', ref_points)
np.save('/data/street/gt.npy', aug_transform)

In [5]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a

t,r,a

(15642329088, 0, 0)